In [3]:
import customtkinter as ctk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.utils import ImageReader
import os
import datetime

class CancerClassificationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🩺 Cancer Detection AI")
        self.root.geometry("900x750")
        ctk.set_appearance_mode("dark")  # "light" or "system"
        ctk.set_default_color_theme("blue")  # "dark-blue", "green", etc.
        
        # Charger le modèle
        try:
            self.model = load_model(r"C:/Users/HP/Documents/projet IA/my_model/my_model.h5")
            print("✅ Modèle chargé avec succès")
        except Exception as e:
            messagebox.showerror("Erreur", f"Impossible de charger le modèle: {str(e)}")
            self.model = None
        
        # Variables
        self.image_path = None
        self.prediction_result = None
        self.confidence = None
        self.detailed_probs = None
        self.class_names = ['colon_aca', 'colon_n', 'lung_aca', 'lung_n', 'lung_scc']
        self.class_names_fr = {
            'colon_aca': 'Adénocarcinome du colon',
            'colon_n': 'Colon normal',
            'lung_aca': 'Adénocarcinome du poumon',
            'lung_n': 'Poumon normal',
            'lung_scc': 'Carcinome épidermoïde du poumon'
        }
        
        self.interpretations = {
            'colon_aca': "L'adénocarcinome du côlon est le type le plus courant de cancer du côlon. Une détection précoce est cruciale pour un traitement efficace.",
            'colon_n': "Le tissu du côlon apparaît normal sans signe de malignité.",
            'lung_aca': "L'adénocarcinome du poumon est le type le plus courant de cancer du poumon chez les non-fumeurs.",
            'lung_n': "Le tissu pulmonaire apparaît normal sans signe de malignité.",
            'lung_scc': "Le carcinome épidermoïde du poumon est souvent associé au tabagisme."
        }
        
        self.create_widgets()
    
    def create_widgets(self):
        # Title
        title_label = ctk.CTkLabel(self.root, text="🩺 Cancer Detection AI",
                font=ctk.CTkFont(size=28, weight="bold"))
        title_label.pack(pady=20)
        
        subtitle = ctk.CTkLabel(self.root, text="Analyse automatique d'images médicales",
                font=ctk.CTkFont(size=16))
        subtitle.pack(pady=(0, 15))

        # Image Frame
        self.image_frame = ctk.CTkFrame(self.root, width=500, height=300, corner_radius=15)
        self.image_frame.pack(pady=15)
        self.image_frame.pack_propagate(False)
        
        self.image_label = ctk.CTkLabel(self.image_frame, text="Aucune image sélectionnée")
        self.image_label.pack(expand=True)

        # Buttons
        button_frame = ctk.CTkFrame(self.root, fg_color="transparent")
        button_frame.pack(pady=15)
        
        load_btn = ctk.CTkButton(button_frame, text="📂 Charger une image", command=self.load_image)
        load_btn.grid(row=0, column=0, padx=15)
        
        predict_btn = ctk.CTkButton(button_frame, text="🔎 Analyser l'image", fg_color="green", command=self.predict)
        predict_btn.grid(row=0, column=1, padx=15)
        
        pdf_btn = ctk.CTkButton(button_frame, text="📄 Générer rapport PDF", fg_color="red", command=self.generate_pdf)
        pdf_btn.grid(row=0, column=2, padx=15)

        # Result Frame
        result_frame = ctk.CTkFrame(self.root, corner_radius=15)
        result_frame.pack(pady=20, fill="both", expand=True, padx=20)
        
        result_title = ctk.CTkLabel(result_frame, text="Résultats de la prédiction",
                               font=ctk.CTkFont(size=18, weight="bold"))
        result_title.pack(pady=10)
        
        self.result_text = ctk.CTkTextbox(result_frame, height=300, width=700, font=ctk.CTkFont(size=14))
        self.result_text.pack(pady=10, padx=15, fill="both", expand=True)
    
    def load_image(self):
        file_path = filedialog.askopenfilename(
            title="Sélectionner une image",
            filetypes=[("Images", "*.jpg *.jpeg *.png *.bmp *.tif *.tiff")]
        )
        
        if file_path:
            self.image_path = file_path
            try:
                img = Image.open(file_path)
                img.thumbnail((450, 300))
                photo = ImageTk.PhotoImage(img)
                self.image_label.configure(image=photo, text="")
                self.image_label.image = photo
            except Exception as e:
                messagebox.showerror("Erreur", f"Impossible de charger l'image: {str(e)}")
    
    def preprocess_image(self, img_path):
        try:
            input_shape = self.model.input_shape[1:3]  # ex: (224, 224)
            img = Image.open(img_path).convert("RGB").resize(input_shape)
            img_array = np.array(img) / 255.0
            img_array = np.expand_dims(img_array, axis=0)
            return img_array
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur de prétraitement: {str(e)}")
            return None
    
    def predict(self):
        if not self.image_path:
            messagebox.showwarning("Avertissement", "Veuillez d'abord charger une image.")
            return
        if not self.model:
            messagebox.showerror("Erreur", "Modèle non chargé.")
            return
        
        try:
            processed_image = self.preprocess_image(self.image_path)
            if processed_image is None:
                return
            
            predictions = self.model.predict(processed_image)
            predicted_class_index = np.argmax(predictions, axis=1)[0]
            confidence = np.max(predictions) * 100
            
            class_name = self.class_names[predicted_class_index]
            class_name_fr = self.class_names_fr.get(class_name, class_name)
            
            self.prediction_result = class_name_fr
            self.confidence = confidence
            self.detailed_probs = []
            
            for i, prob in enumerate(predictions[0]):
                cls_name = self.class_names[i]
                cls_name_fr = self.class_names_fr.get(cls_name, cls_name)
                self.detailed_probs.append((cls_name_fr, prob*100))
            
            # Display
            self.result_text.delete("1.0", "end")
            self.result_text.insert("end", f"✅ Classe prédite: {class_name_fr}\n")
            self.result_text.insert("end", f"🎯 Confiance: {confidence:.2f}%\n\n")
            self.result_text.insert("end", "📊 Probabilités détaillées:\n")
            for cls_name_fr, prob in self.detailed_probs:
                self.result_text.insert("end", f"   - {cls_name_fr}: {prob:.2f}%\n")
            
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur lors de la prédiction: {str(e)}")
    
    def generate_pdf(self):
        if not self.image_path or not self.prediction_result:
            messagebox.showwarning("Avertissement", "Veuillez d'abord analyser une image.")
            return
        
        try:
            file_path = filedialog.asksaveasfilename(
                defaultextension=".pdf",
                filetypes=[("Fichiers PDF", "*.pdf")],
                title="Enregistrer le rapport PDF"
            )
            if not file_path:
                return  
            
            c = canvas.Canvas(file_path, pagesize=letter)
            width, height = letter
            
            c.setFont("Helvetica-Bold", 16)
            c.drawString(100, height - 50, "Rapport d'Analyse - Détection de Cancer")
            c.setFont("Helvetica", 12)
            c.drawString(100, height - 70, f"Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}")
            
            img = Image.open(self.image_path)
            img.thumbnail((300, 300))
            img_path_temp = "temp_img.jpg"
            img.save(img_path_temp)
            c.drawImage(ImageReader(img_path_temp), 100, height - 400, width=300, height=300)
            
            c.setFont("Helvetica-Bold", 14)
            c.drawString(100, height - 420, "Résultats de l'analyse:")
            c.setFont("Helvetica", 12)
            c.drawString(100, height - 440, f"Classe prédite: {self.prediction_result}")
            c.drawString(100, height - 460, f"Niveau de confiance: {self.confidence:.2f}%")
            
            c.drawString(100, height - 490, "Probabilités détaillées:")
            y_pos = height - 510
            for cls_name, prob in self.detailed_probs:
                c.drawString(120, y_pos, f"{cls_name}: {prob:.2f}%")
                y_pos -= 20
            
            c.setFont("Helvetica-Bold", 14)
            c.drawString(100, y_pos - 30, "Interprétation:")
            c.setFont("Helvetica", 12)
            
            interpretation_key = None
            for key, value in self.class_names_fr.items():
                if value == self.prediction_result:
                    interpretation_key = key
                    break
            
            if interpretation_key and interpretation_key in self.interpretations:
                interpretation = self.interpretations[interpretation_key]
                lines = []
                words = interpretation.split()
                line = ""
                for word in words:
                    if len(line) + len(word) < 80:
                        line += word + " "
                    else:
                        lines.append(line)
                        line = word + " "
                if line:
                    lines.append(line)
                
                y_pos -= 50
                for line in lines:
                    c.drawString(100, y_pos, line)
                    y_pos -= 20
            
            c.setFont("Helvetica-Oblique", 10)
            c.drawString(110, 60, "⚠️ Ce rapport a été généré automatiquement par l'application Cancer Detection AI.")
            c.drawString(120, 45, "Ces résultats doivent être validés par un professionnel de santé qualifié.")
            
            c.save()
            os.remove(img_path_temp)
            
            messagebox.showinfo("Succès", f"📄 Rapport PDF généré avec succès:\n{file_path}")
            
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur lors de la génération du PDF: {str(e)}")

if __name__ == "__main__":
    app = CancerClassificationApp(ctk.CTk())
    app.root.mainloop()


✅ Modèle chargé avec succès
